*cold start with input keywords  <br>
*scale after split

In [1]:
# %pip install scikit-surprise

In [1]:
import numpy as np
import pandas as pd
import time
from surprise import SVD, SVDpp
from surprise import SlopeOne, BaselineOnly
from surprise import accuracy
from surprise import Reader
from surprise import Dataset
from sklearn.preprocessing import MinMaxScaler
from surprise import KNNBaseline, KNNWithZScore,KNNWithMeans
from surprise.model_selection import cross_validate
from sklearn.model_selection import train_test_split

In [2]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [3]:
ratings_df = pd.read_csv('C:/Users/User/Documents/#M.L/2023/Final Project ML 2023 Recommender Systems/Datasets/ml-latest-small/ratings.csv')
movies_df  = pd.read_csv('C:/Users/User/Documents/#M.L/2023/Final Project ML 2023 Recommender Systems/Datasets/ml-latest-small/movies.csv')

In [ ]:
# ratings_df = pd.read_table('C:/Users/User/Documents/#M.L/2023/Final Project ML 2023 Recommender Systems/Datasets/ml-1m/ratings.dat', delimiter="::", engine='python', header=None, names=['userId','movieId','rating','timestamp'])
# movies_df  = pd.read_table('C:/Users/User/Documents/#M.L/2023/Final Project ML 2023 Recommender Systems/Datasets/ml-1m/movies.dat' , encoding='latin-1', delimiter="::", engine='python', header=None, names=['movieId','title','genres'])

In [6]:
# ratings_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ml-10M/ratings.dat',delimiter="::",engine='python', header=None, names=['userId', 'movieId', 'rating', 'timestamp'])
# movies_df  = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ml-10M/movies.dat' ,delimiter="::",engine='python', header=None, names=['movieId', 'title', 'genres'])

In [7]:
# ratings_df = pd.read_csv('C:/Users/User/Documents/#M.L/2023/Final Project ML 2023 Recommender Systems/Datasets/ml-latest/ratings.csv')
# movies_df  = pd.read_csv('C:/Users/User/Documents/#M.L/2023/Final Project ML 2023 Recommender Systems/Datasets/ml-latest/movies.csv')

In [10]:
complete_dataset = pd.merge(ratings_df,movies_df,on='movieId')

### Converting pandas DataFrame to Surprise DataFrame

In [10]:
# split the dataset
# trainset_df, testset_df = complete_dataset.loc[complete_dataset.userId != "1"],complete_dataset.loc[complete_dataset.userId == "1"]
# trainset_df, testset_df = train_test_split(complete_dataset, test_size=0.1)

# create a dummy reader
reader = Reader()

# transform the two datasets
data_sdf = Dataset.load_from_df(complete_dataset[['userId', 'movieId', 'rating']], reader)

In [11]:
data = data_sdf.build_full_trainset()

### Calculating the nesesary bias lists for the biased SVD


### K-NN


In [12]:
import time
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms.knns import KNNBaseline, KNNWithZScore, KNNWithMeans

# Define k-NN models
knn_models = [
    KNNBaseline(k=50, sim_options={'name': 'pearson', 'user_based': True}, user_based=False),
    KNNWithZScore(k=50, sim_options={'name': 'pearson', 'user_based': True}, user_based=False),
    KNNWithMeans(k=50, sim_options={'name': 'pearson', 'user_based': True}, user_based=False),
]

knn_model_names = ["k-NN Baseline", "k-NN Z-Score", "k-NN Means"]

# Loop through k-NN models
for model, model_name in zip(knn_models, knn_model_names):
    print(f"*******************    {model_name}   *******************")
    start = time.time()
    cross_validate(model, data_sdf, measures=['RMSE', 'MAE'], cv=10, verbose=True)
    end = time.time()
    model_time = end - start
    print(f" {model_name} Cross Validation time {model_time}")


*******************    k-NN Baseline   *******************


Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als.

In [13]:
sp1 = SlopeOne()

start = time.time()

sp1.fit(data)
end = time.time()
print(f' Baseline SVD Training time {end-start} sec')

 Baseline SVD Training time 8.779996156692505 sec


### Baseline Only
its a basic prediction algorythm, we use it as a base to calculate the improvements of the rest algorithms upon.

In [14]:
baseline = BaselineOnly(verbose=False)

start = time.time()

baseline.fit(data)
end = time.time()
print(f' Baseline SVD Training time {end-start} sec')

 Baseline SVD Training time 0.36199450492858887 sec


### Single Variable Decomposition
3 viariatios upon the popular matrix factorisation algorithm.

In [15]:
svd = SVD(verbose=False, n_epochs=20, random_state=seed)

start = time.time()

svd.fit(data)
end = time.time()
print(f' Baseline SVD Training time {end-start} sec')

 Baseline SVD Training time 1.9139907360076904 sec


In [16]:
svdpp = SVDpp(verbose=False, n_epochs=20, random_state=seed, cache_ratings=False)

start = time.time()

svdpp.fit(data)
end = time.time()
print(f' Baseline SVD Training time {end-start} sec')

 Baseline SVD Training time 330.7479965686798 sec


In [17]:
svdppc = SVDpp(verbose=False, n_epochs=20, random_state=seed, cache_ratings=True)

start = time.time()

svdppc.fit(data)
end = time.time()
print(f' Baseline SVD Training time {end-start} sec')

 Baseline SVD Training time 245.40833234786987 sec


In [18]:
models = [sp1, baseline, svd, svdpp, svdppc]
model_names = ["SlopeOne", "BaselineOnly", "Base SVD", "SVDpp", "Cached SVDpp"]

for model, model_name in zip(models, model_names):
    print(f" *******************   {model_name}    *******************")
    start = time.time()
    cross_validate(model, data_sdf, measures=['RMSE', 'MAE'], cv=10, verbose=True)
    end = time.time()
    print(f" {model_name} Cross Validation time {end - start}")

 *******************   SlopeOne    *******************
Evaluating RMSE, MAE of algorithm SlopeOne on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.8975  0.8942  0.8942  0.9012  0.8936  0.8844  0.8950  0.8826  0.9050  0.9011  0.8949  0.0067  
MAE (testset)     0.6866  0.6861  0.6824  0.6848  0.6857  0.6751  0.6838  0.6778  0.6865  0.6842  0.6833  0.0037  
Fit time          8.15    7.17    7.44    7.16    7.39    7.79    7.89    7.34    7.43    7.31    7.51    0.31    
Test time         4.72    4.36    4.69    4.52    4.40    4.31    4.44    4.38    4.56    4.45    4.48    0.13    
 SlopeOne Cross Validation time 122.3729977607727
 *******************   BaselineOnly    *******************
Evaluating RMSE, MAE of algorithm BaselineOnly on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.86

## Autoencoder

In [19]:
# %pip install -q tensorflow-model-optimization

In [4]:
print(ratings_df.rating.max(), ratings_df.rating.min())

5.0 0.5


In [7]:
ratings_df.drop(columns=["timestamp"], inplace=True)

KeyError: "['timestamp'] not found in axis"

In [11]:
matrix_df = ratings_df.pivot(index='userId', columns='movieId', values='rating')

matrix_df = matrix_df.reset_index()
matrix_df.drop(columns=["userId"], inplace=True)

In [13]:
print(len(ratings_df.userId.unique()),
      len(ratings_df.movieId.unique()),
      len(movies_df.movieId.unique()),
      matrix_df.shape)

610 9724 9742 (610, 9724)


In [14]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

ratings_to_scale = matrix_df.iloc[:, 0:]

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_ratings = scaler.fit_transform(ratings_to_scale)

matrix_df.iloc[:, 0:] = scaled_ratings

train_df, test_df = train_test_split(matrix_df, test_size=0.2, random_state=42)
n_users = len(ratings_df.userId.unique())
n_movies = len(ratings_df.movieId.unique())

full_index_values  = matrix_df.index

In [15]:
nan_replace = 0.5

train_mask = ~np.isnan(train_df)
test_mask = ~np.isnan(test_df)
full_mask = ~np.isnan(matrix_df)

train = train_df.fillna(nan_replace)
test = test_df.fillna(nan_replace)
full = matrix_df.fillna(nan_replace)

train_np = train.to_numpy()
test_np  = test.to_numpy()
full_np = full.to_numpy()

In [16]:
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Custom Loss Function to calculate loss only for the existing data and not the data that we filled above

In [17]:
from keras.losses import mean_squared_error
from keras import backend

def masked_mse(y_true, y_pred):
    mask = backend.cast(backend.not_equal(y_true, 0), backend.floatx())
    return mean_squared_error(y_true * mask, y_pred * mask)

In [18]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense
import numpy as np

# Create the autoencoder
# creat the encoder
encoder_in = Input(shape=(full_np.shape[1],))
encoder1 = Dense(255, activation='relu')(encoder_in)
# encoder2 = Dense(128, activation='relu')(encoder1)
encoder_out = Dense(64, activation='linear')(encoder1)
encoderM = Model(inputs=encoder_in, outputs=encoder_out)
encoderM.compile(optimizer='adam', loss=masked_mse)


# creat the decoder
decoder_in = Input(shape=(64,))
decoder1 = Dense(64, activation='relu')(decoder_in)
# decoder2 = Dense(128, activation='relu')(decoder1)
decoder_out = Dense(n_movies, activation='linear')(decoder1)
decoderM = Model(inputs=decoder_in, outputs=decoder_out)
decoderM.compile(optimizer='adam', loss=masked_mse)


autoencoder = Model(inputs=encoder_in, outputs=decoderM(encoderM(encoder_in)))
autoencoder.compile(optimizer='adam', loss=masked_mse)


Compiling with the costom loss function

In [19]:
autoencoder.fit(full_np, full_np, epochs=10, batch_size=32, shuffle=True)

Epoch 1/10

20/20 [==============================] - 4s 64ms/step - loss: 0.0701
Epoch 2/10
20/20 [==============================] - 1s 59ms/step - loss: 0.0032
Epoch 3/10
20/20 [==============================] - 1s 66ms/step - loss: 0.0017
Epoch 4/10
20/20 [==============================] - 1s 60ms/step - loss: 0.0013
Epoch 5/10
20/20 [==============================] - 1s 59ms/step - loss: 0.0013
Epoch 6/10
20/20 [==============================] - 1s 62ms/step - loss: 0.0013
Epoch 7/10
20/20 [==============================] - 1s 63ms/step - loss: 0.0013
Epoch 8/10
20/20 [==============================] - 1s 61ms/step - loss: 0.0013
Epoch 9/10
20/20 [==============================] - 1s 60ms/step - loss: 0.0014
Epoch 10/10
20/20 [==============================] - 1s 70ms/step - loss: 0.0013


### Tesing prooning


In [14]:
# from tensorflow_model_optimization.python.core.sparsity.keras import pruning_schedule
# from tensorflow_model_optimization.python.core.sparsity.keras import pruning
# from tensorflow_model_optimization import update_pruning_state

# # Assuming you have already defined and compiled your autoencoder, encoder, and decoder

# # Create a pruning schedule (you can customize the parameters)
# pruning_schedule = pruning_schedule.ConstantSparsity(0.5, begin_step=0, frequency=100)

# # Define a pruning callback for the encoder
# pruning_callback = pruning.UpdatePruningStep()

# # Prune the encoder
# pruned_encoder = prune.prune_low_magnitude(encoderM, pruning_schedule=pruning_schedule)

# # Re-compile the pruned encoder
# pruned_encoder.compile(optimizer='adam', loss=mean_squared_error)

# # Train the pruned encoder
# pruned_encoder.fit(x_train, x_train, epochs=10, batch_size=32, validation_data=(x_val, x_val))

# # Strip pruning wrappers to get the final pruned encoder
# pruned_encoder = prune.strip_pruning(pruned_encoder)

# # Now you can use the pruned encoder in your autoencoder
# autoencoder_in = Input(shape=(full_np.shape[1],))
# encoded_representation = pruned_encoder(autoencoder_in)
# decoded_representation = decoderM(encoded_representation)
# pruned_autoencoder = Model(inputs=autoencoder_in, outputs=decoded_representation)

# # Compile the pruned autoencoder
# pruned_autoencoder.compile(optimizer='adam', loss=mean_squared_error)


### Trying custom k-Folds method

In [15]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend
import time

# Define masked MSE and RMSE functions
def masked_mse(y_true, y_pred):
    mask = backend.cast(backend.not_equal(y_true, 0), backend.floatx())
    return mean_squared_error(y_true * mask, y_pred * mask)

def masked_rmse(y_true, y_pred):
    mse = masked_mse(y_true, y_pred)
    return np.sqrt(mse)

def masked_mae(y_true, y_pred):
    mask = backend.cast(backend.not_equal(y_true, 0), backend.floatx())
    return mean_absolute_error(y_true * mask, y_pred * mask)

num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
kf.get_n_splits(full_np)

# Placeholder for results
mae_scores = []
rmse_scores = []
training_times = []

# Perform K-Fold cross-validation
for fold, (train_index, test_index) in enumerate(kf.split(full_np), 1):
    X_train, X_test = full_np[train_index], full_np[test_index]

    # Record start time
    start_time = time.time()

    # Make predictions
    y_pred = autoencoder.predict(X_test)

    # Record end time
    end_time = time.time()


    # Calculate MSE and RMSE for this fold with custom masking
    mse_fold = masked_mae(X_test, y_pred)
    rmse_fold = masked_rmse(X_test, y_pred)

    mae_scores.append(mse_fold)
    rmse_scores.append(rmse_fold)
    training_times.append(end_time - start_time)

    # Print MSE, RMSE, and training time for this fold
    print(f"Fold {fold}: MAE = {mse_fold:.4f}, RMSE = {rmse_fold:.4f}, Testing Time = {end_time - start_time:.2f} seconds")

# Calculate and print the mean and standard deviation for MSE, RMSE, and training times
mean_mse = np.mean(mae_scores)
std_mse = np.std(mae_scores)
mean_rmse = np.mean(rmse_scores)
std_rmse = np.std(rmse_scores)
mean_training_time = np.mean(training_times)
std_training_time = np.std(training_times)

print(f"\nMean MAE across {num_folds} folds: {mean_mse:.4f} ± {std_mse:.4f}")
print(f"Mean RMSE across {num_folds} folds: {mean_rmse:.4f} ± {std_rmse:.4f}")
print(f"Mean Testing Time across {num_folds} folds: {mean_training_time:.2f} ± {std_training_time:.2f} seconds")


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


NotImplementedError: in user code:

    File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\User\AppData\Local\Temp\ipykernel_8620\1917560111.py", line 7, in masked_mse  *
        return mean_squared_error(y_true * mask, y_pred * mask)
    File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\_param_validation.py", line 435, in wrapper  *
        return func(*args, **kwargs)
    File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_regression.py", line 473, in mean_squared_error  *
        y_type, y_true, y_pred, multioutput = _check_reg_targets(
    File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_regression.py", line 99, in _check_reg_targets  *
        check_consistent_length(y_true, y_pred)
    File "C:\Users\User\AppData\Roaming\Python\Python311\site-

In [20]:
predictions = autoencoder.predict(full_np)

20/20 [==============================] - 0s 14ms/step


In [21]:
newMovie_predictions = predictions.copy()
newMovie_predictions[full_mask] = np.nan # Make already wached movies prediction Nan 
column_names = movies_df['title'].values.tolist()[:(newMovie_predictions.shape[1])]
newMovie_predictions_df = pd.DataFrame(data=newMovie_predictions, columns=column_names)
newMovie_predictions_df.set_index(pd.Index(full_index_values), inplace=True)
newMovie_predictions_T =newMovie_predictions_df.T
# newMovie_predictions_T.head()

In [17]:
while True:
    try:
        userID = int(input("Enter userID: "))
        print("You entered:", userID)

        # Check if userID is in the list
        if userID in full_index_values:
            print("UserID is in the list.")
            break  # Break the loop if a valid ID is entered
        else:
            print("UserID is not in the list. Please try again.")

    except ValueError:
        print("Invalid input. Please enter a valid integer.")

You entered: 100
UserID is in the list.


In [18]:
user_rest = newMovie_predictions_df.index.difference([userID])
top_n = 10  # Number of top recommendations

newMovie_predictions_user = newMovie_predictions_T.drop(user_rest, axis=1, inplace=False)
newMovie_predictions_user = newMovie_predictions_user.dropna()

newMovie_predictions_user.sort_values(by=userID, ascending=False)[:top_n]*5

,100
"Shawshank Redemption, The (1994)",3.249163
Forrest Gump (1994),3.149484
"Silence of the Lambs, The (1991)",3.128417
Pulp Fiction (1994),3.111966
"Walk on the Moon, A (1999)",2.994485
Star Wars: Episode IV - A New Hope (1977),2.991528
Jurassic Park (1993),2.987076
Braveheart (1995),2.956501
Toy Story (1995),2.945926
"Princess Bride, The (1987)",2.927420
